In [2]:
import math
import os
import random
import numpy as np
import skimage
import scipy
import pickle

import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics

from sklearn.model_selection import train_test_split

In [3]:
os.chdir('../')

In [4]:
with open('../xtrain.npy','rb') as f:
    x = np.load('../xtrain.npy')

In [5]:
X_train, X_test = train_test_split(x, random_state=False, test_size=0.2, shuffle=True)

In [6]:
#Take the example from the newly created numpy dataset
first_example = x[0]
first_example_image = tf.keras.preprocessing.image.array_to_img(first_example.reshape(224,224,3))
first_example_image.show()

In [9]:
original_image = tf.keras.preprocessing.image.load_img("../images/Set3/buffalo/001.jpg")
original_image_to_array = tf.keras.preprocessing.image.img_to_array(original_image)
reshaped_image = original_image_to_array.reshape(-1,3)/255.0

In [17]:
#Using KMeans Algorithm for Image Segmentation
X_train= x.reshape(-1,3)
kmeans = KMeans(n_clusters = 4,random_state=1).fit(X_train)
pickle.dump(kmeans,open('models/four_clusters','wb'))

In [20]:
X_test.dtype

dtype('float64')

In [39]:
kmeans = pickle.load(open('../models/four_clusters','rb'))
#buffer data type mismatch fixed by casting X_Test.dtype to float32 which is same as that of kmeans.cluster_centers_.dtype
training_images = X_test[np.random.permutation(225)[:10]].astype("f") 
num = 0
for i in training_images:
    prediction = kmeans.predict(i)
    recovered_image = kmeans.cluster_centers_[prediction,:]
    recovered_image_array = np.reshape(recovered_image,(224,224,3))*255.0
    recovered_image = tf.keras.preprocessing.image.array_to_img(recovered_image_array)
    recovered_image.save(f"../images/Set2/KMeans Predictions/Prediction {num}.JPG")
    num = num + 1 

D:\COMP303\venv\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator KMeans from version 1.0.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [40]:
distortions = []
for i in range(1, 11):
    km = KMeans(
        n_clusters=i, init='random',
        n_init=10, max_iter=300,
        tol=1e-04, random_state=0
    )
    km.fit(reshaped_dataset)
    distortions.append(km.inertia_)

# plot
plt.plot(range(1, 4), distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.show()